In [1]:
import pandas as pd
import numpy as np
import math
from nptdms import TdmsFile
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# File paths
file_path = "/Users/shubhangchauhan/Desktop/Procyon Data/TaehwaV3/11_06_2025/10nmSweep1_11_06_2020_12_13 0.tdms"
csv_output_path = "/Users/shubhangchauhan/Desktop/Your_File_Selected_Data.csv"
html_output_path = "/Users/shubhangchauhan/Desktop/single_file_selected_grid.html"

tdms_file = TdmsFile.read(file_path)
df = tdms_file.as_dataframe()
df.columns = df.columns.str.replace(" ", "_")
df.columns = df.columns.str.replace("/'Calculated_Data'/", "", regex=True)
df.columns = df.columns.str.replace("'", "", regex=False)
df["Index"] = range(len(df))

# List of individual variables to plot
single_var_plots = ["CAN_Id_in", "CAN_Iq_in", "CAN_Vstator_V", "CAN_Vd_Out", "CAN_Vq_Out","CAN_Istator_A","CAN_Id_Ref","CAN_Iq_Ref"]

# List of pairs to overlay in one subplot: (var1, var2)
dual_var_plots = [
    ("Torque", "Speed") #,
    #("CAN_Istator_A", "EFFI1")
]

# Plotting
total_plots = len(single_var_plots) + len(dual_var_plots)
cols_per_row = 2
rows = math.ceil(total_plots / cols_per_row)

titles = [col for col in single_var_plots] + [f"{a} + {b}" for a, b in dual_var_plots]
fig = make_subplots(
    rows=rows,
    cols=cols_per_row,
    subplot_titles=titles,
    specs=[[{"secondary_y": True} for _ in range(cols_per_row)] for _ in range(rows)]
)

plot_idx = 0

# Plot single variable plots
for col in single_var_plots:
    if col in df.columns:
        row, col_idx = divmod(plot_idx, cols_per_row)
        fig.add_trace(go.Scatter(x=df["Index"], y=pd.to_numeric(df[col], errors="coerce"),
                                 name=col, mode="lines"),
                      row=row+1, col=col_idx+1)
        plot_idx += 1

# Plot dual variable overlays
for var1, var2 in dual_var_plots:
    row, col_idx = divmod(plot_idx, cols_per_row)
    row += 1
    col_idx += 1
    yaxis_base = f"y{(row - 1) * cols_per_row + col_idx}"
    yaxis_secondary = f"y{(row - 1) * cols_per_row + col_idx}2"

    # Primary Y-axis trace
    if var1 in df.columns:
        fig.add_trace(go.Scatter(
            x=df["Index"],
            y=pd.to_numeric(df[var1], errors="coerce"),
            name=var1,
            mode="lines",
            yaxis=yaxis_base
        ), row=row, col=col_idx,secondary_y=False)

    # Secondary Y-axis trace
    if var2 in df.columns:
        fig.add_trace(go.Scatter(
            x=df["Index"],
            y=pd.to_numeric(df[var2], errors="coerce"),
            name=var2,
            mode="lines",
            yaxis=yaxis_secondary
        ), row=row, col=col_idx,secondary_y=True)

    fig.update_yaxes(title_text=var1, row=row, col=col_idx)
    fig.update_yaxes(title_text=var2, row=row, col=col_idx, secondary_y=True)
    plot_idx += 1


fig.update_layout(
    height=200 * rows,
    width=720 * cols_per_row,
    title=f"Custom Data Subplots: {file_path.split('/')[-1]}",
    showlegend=False
)

# Save outputs
df_out_cols = ["Index"] + list(set(single_var_plots + [x for pair in dual_var_plots for x in pair]))
df[df_out_cols].to_csv(csv_output_path, index=False)
fig.write_html(html_output_path)

print("Saved CSV and HTML plot.")


Saved CSV and HTML plot.
